In [4]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from model import Model
from IPython import display
import cv2

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15, 10)


In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.empty_cache()

In [5]:
model = Model(use_multiple_gpu=True, batch_size=8, val_dataset_dir='data_scene_flow_small/testing', train_dataset_dir='data_scene_flow_small/training')

fig, ax = plt.subplots(ncols=1, nrows=2)
epochs = 50

for epoch in range(epochs):
    model.train(epoch)
    fig.suptitle('Epoch {}'.format(epoch))
    d, ref_img = model.test()
    resized_d0 = cv2.resize(d[0], (1242, 375), interpolation=cv2.INTER_LINEAR)
    resized_i0 = cv2.resize(np.moveaxis(ref_img, 0, -1), (1242, 375), interpolation=cv2.INTER_LINEAR)
    ax[0].imshow(resized_i0)
    ax[1].imshow(resized_d0, cmap='plasma')
    
    display.display(plt.gcf())

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
# d, _ = model.test()
# np.save(path.join(self.output_directory, 'disparities.npy'), d)